# LAB9

In [353]:
from random import random, choice, randint

import lab9_lib

In [354]:
from dataclasses import dataclass
from copy import copy
from math import ceil

In [355]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 80
TOURNAMENT_SIZE = min(70, POPULATION_SIZE) # increase to increase selective pressure
MUTATION_PROBABILITY = .15

MUTATION_REPETITION = .05

LOCI_NUMBER = 1000

In [356]:
@dataclass
class Individual:
    fitness: tuple
    genotype: list[bool]

    def __str__(self):
        return f"{self.fitness:.2%} " + "".join([str(i) for i in [int(b) for b in self.genotype]])

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, LOCI_NUMBER-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, LOCI_NUMBER-1)
    offspring = Individual(
        genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:],
        fitness=None)
    assert len(offspring.genotype) == LOCI_NUMBER
    return offspring

def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    offspring = Individual(
        genotype=[choice([ind1.genotype[i], ind2.genotype[i]]) for i in range(len(ind1.genotype))],
        fitness=None
    )
    assert len(offspring.genotype) == LOCI_NUMBER
    return offspring

In [357]:
fitness = lab9_lib.make_problem(1)
# fitness = lab9_lib.make_problem(2)
# fitness = lab9_lib.make_problem(5)
# fitness = lab9_lib.make_problem(10)

In [358]:
population = [Individual(
    genotype=[choice((False, False)) for _ in range(LOCI_NUMBER)],
    fitness=None
) for _ in range(POPULATION_SIZE)]

for i in population:
    i.fitness = fitness(i.genotype)

# Island-based diversity promotion

In [359]:
ISLAND_NUMBER = 7
MIGRATION_PROBABILITY = 0.05
ISLANDS_WEIGHT = [1 / ISLAND_NUMBER] * ISLAND_NUMBER # ??

SLICE_SIZE = ceil(POPULATION_SIZE / ISLAND_NUMBER)
MIGRATION_SIZE = min(5, SLICE_SIZE)

In [360]:
SLICE_SIZE

15

In [361]:
islands = [population[SLICE_SIZE*i : SLICE_SIZE*(i+1)] for i in range(min(ISLAND_NUMBER, len(population)))]

In [362]:
def migrate(islands: list):
  for _ in range(MIGRATION_SIZE):
    for i in range(-1, len(islands)-1):
      tmp = islands[i][randint(0, len(islands[i])-1)]
      islands[i][randint(0, len(islands[i])-1)] = islands[i+1][randint(0, len(islands[i+1])-1)]
      islands[i+1][randint(0, len(islands[i+1])-1)] = tmp

In [363]:
for generation in range(5000):
    island = islands[randint(0, ISLAND_NUMBER-1)]

    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:
            p = select_parent(island)
            o = mutate(p)
            """ while random() < MUTATION_REPETITION:
                o = mutate(o) """
        else:
            p1 = select_parent(island)
            p2 = select_parent(island)
            o = uniform_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    island.extend(offspring)
    island.sort(key=lambda i: i.fitness, reverse=True)
    island = island[:SLICE_SIZE]

    if random() < MIGRATION_PROBABILITY:
        migrate(islands)

for island in islands:
    print(island[0])

79.40% 101101010111011110101111110111101110011111111011101111111111111011111111110111110111111110110111001111111110011111110111111111001111101110000100010111111111111111101111010111111110011101011111001111111011111111110111101011011011111101101101011101111111011111111111111111111000011111101110011111101111101111011110111110110111011011101011111111111111011111001100111101110110010111011100111111100111111111111011110110111011111110101111101101111100101111101110110010011111011111111101111011111111111101101011101110111010111111101111111111111111101011111111111011111111111111101101001111110011111110100111110111111111110010111111111111111101111110100010011111111011111111111011111000111111111111000101110111111111100011010111111111011111111100111111111111101011100111111101111110111110111010111111110111111110111111010111101011001110111111111011110111110111111011111101111101111100110111001111111011111100111111111111111101110011011010101111111110111011111111110111011011101110111111101111111111111

In [364]:
fitness.calls

400100